# Coil Data Preparation

In [1]:
# Starting point:
#   data_norm.pkl :  
#       Row1 .- struct scale ZnMin and ZnMax
#       Row2 .- dosD  DB table
#       Row3 .- coils DB table
#       Row4 .- nclmaps (struct of coils with specific labels nzn and 


In [2]:
import sys, os, datetime, pickle, time
import string, pdb, tqdm
import random, cv2, keras, os.path
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from functions import *
from models import *
#
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
#
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
#
from timeit import default_timer as timer
from datetime import datetime, timedelta, date
from PIL import Image
from IPython.display import Image
# %matplotlib inline

#
tf.autograph.set_verbosity(0)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
# config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# tf.get_logger().setLevel('ERROR')

c:\Users\alexm\AppData\Local\Programs\Python\Python39\lib\site-packages\evidently\analyzers\__init__.py:3: UserWarning: analyzers are deprecated, use metrics instead
  warnings.warn("analyzers are deprecated, use metrics instead")
c:\Users\alexm\AppData\Local\Programs\Python\Python39\lib\site-packages\evidently\dashboard\__init__.py:8: UserWarning: dashboards are deprecated, use metrics instead
  warnings.warn("dashboards are deprecated, use metrics instead")


[]


# Preparing Data

In [3]:
# Obtener la ruta del directorio tres niveles arriba
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))

# Cargar datasets desde el directorio tres niveles arriba
data_norm_path = os.path.join(parent_dir, 'Dataset/data_norm.pkl')

with open(data_norm_path, 'rb') as handle:
    GlblFrm = pickle.load(handle)
    dosD    = pickle.load(handle)
    coils   = pickle.load(handle)
    nclmaps = pickle.load(handle)


In [4]:
GlblFrm

{'NormMn': -279.38157, 'NormMx': 507.54791, 'MaxTiles': 264}

In [5]:
dosD

,COILID,TILEID,MID,PLANT,SIDE,MEAN,MAX,MIN,COUNT
0,225216688,0.0,1234,1066.0,T,147.09500,147.09500,147.09500,1.0
1,225216688,28.0,1234,1066.0,T,143.53551,143.53551,143.53551,1.0
2,225216688,56.0,1234,1066.0,T,132.94450,132.94450,132.94450,1.0
3,225216688,85.0,1234,1066.0,T,141.23540,141.23540,141.23540,1.0
4,225216688,113.0,1234,1066.0,T,143.35880,143.35880,143.35880,1.0
...,...,...,...,...,...,...,...,...,...
1272307,233667056,130929.0,1243,1066.0,T,46.29932,46.29932,46.29932,1.0
1272308,233667056,130958.0,1243,1066.0,T,45.68384,45.68384,45.68384,1.0
1272309,233667056,130986.0,1243,1066.0,T,44.40857,44.40857,44.40857,1.0
1272310,233667056,131015.0,1243,1066.0,T,44.03534,44.03534,44.03534,1.0


In [6]:
num_registros_valores_iguales = len(dosD[(dosD['MEAN'] == dosD['MAX']) & (dosD['MAX'] == dosD['MIN'])])
unicos_dosD = dosD.nunique()

print("Número de registros donde los valores de 'mean', 'max' y 'min' son iguales:", num_registros_valores_iguales)
print("Valores únicos por columna: \n", unicos_dosD)

Número de registros donde los valores de 'mean', 'max' y 'min' son iguales: 1272312
Valores únicos por columna: 
 COILID      1160
TILEID      8846
MID            2
PLANT          1
SIDE           1
MEAN      782728
MAX       782728
MIN       782728
COUNT          1
dtype: int64


In [7]:
coils

,SID,PLANT,NAME,STARTTIME,MATERIAL,LENGTH,WIDTH,THICK,WEIGHT,BASEPID,CLASSLABEL,Label,ZnMin,ZnMax
0,226031456,1066.0,226031456,2022-01-28,0.0,792.0,1183.0,3.0,22.0,220100.0,NOK,0,60,80
1,226101621,1066.0,226101621,2022-01-31,0.0,273.0,1252.0,4.0,10.0,220100.0,NOK,2,215,258
2,226120037,1066.0,226120037,2022-01-31,0.0,599.0,1305.0,2.0,15.0,220100.0,OK,1,155,175
3,225387481,1066.0,225387481,2022-01-13,0.0,268.0,1502.0,5.0,15.0,220100.0,OK,1,60,80
4,225406471,1066.0,225406471,2022-01-13,0.0,735.0,1158.0,2.0,19.0,220100.0,NOK,0,45,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155,233642421,1066.0,233642421,2022-08-31,0.0,576.0,1422.0,3.0,24.0,220800.0,OK,2,138,158
1156,233445180,1066.0,233445180,2022-08-25,0.0,513.0,1437.0,4.0,23.0,220800.0,OK,1,125,145
1157,232624927,1066.0,232624927,2022-08-01,0.0,345.0,1482.0,2.0,11.0,220800.0,OK,2,155,175
1158,232635130,1066.0,232635130,2022-08-01,0.0,457.0,968.0,3.0,11.0,220800.0,OK,1,88,108


In [8]:
unicos_coils = coils.nunique()
unicos_coils

SID           1160
PLANT            1
NAME          1160
STARTTIME      204
MATERIAL         1
LENGTH         638
WIDTH          374
THICK            7
WEIGHT          30
BASEPID          8
CLASSLABEL       2
Label            4
ZnMin           23
ZnMax           25
dtype: int64

In [9]:
coil_keys = list(nclmaps['225216688'].keys())

print("Número de claves en el diccionario: ",len(nclmaps.keys()))
print("Claves por cada bobina: ", coil_keys)

Número de claves en el diccionario:  1160
Claves por cada bobina:  [1234, 1243, 'props']


In [10]:
for i in coil_keys:
    print(list(nclmaps['225216688'][i].keys()))


['pos', 'zn', 'nzn', 'nzne']
['pos', 'zn', 'nzn', 'nzne']
['ntiles', 'Len', 'Wid', 'Label', 'ZnMn', 'ZnMx', 'nsns', 'callib_1234', 'nchks', 'nfltls', 'callib_1243', '1234_mx', '1234_mn', '1243_mx', '1243_mn', 'MxVal', 'MnVal']


In [11]:
print('GlblFrm:',GlblFrm)
print('nclmaps keys:',nclmaps[list(nclmaps.keys())[0]].keys())
print('nclmaps props:',nclmaps[list(nclmaps.keys())[0]]['props'])

GlblFrm: {'NormMn': -279.38157, 'NormMx': 507.54791, 'MaxTiles': 264}
nclmaps keys: dict_keys([1234, 1243, 'props'])
nclmaps props: {'ntiles': 64, 'Len': 638.0, 'Wid': 1108.0, 'Label': 1, 'ZnMn': 138, 'ZnMx': 158, 'nsns': 9, 'callib_1234': {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: []}, 'nchks': {0: 1, 1: 2, 2: 5, 3: 5, 4: 7, 5: 3, 6: 0, 7: 0, 8: 0}, 'nfltls': {0: 4, 1: 4, 2: 44, 3: 42, 4: 30, 5: 4, 6: 0, 7: 0, 8: 0}, 'callib_1243': {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: []}, '1234_mx': 37.35920999999999, '1234_mn': -75.1079, '1243_mx': 54.614810000000006, '1243_mn': -10.365899999999996, 'MxVal': 54.614810000000006, 'MnVal': -75.1079}


In [12]:
coils['Label'].value_counts()

# 1 --> OK
# 2 --> NOK

Label
1    547
0    261
3    212
2    140
Name: count, dtype: int64

# Building Sets

In [13]:
#  Selecting coils OK and NOK
lsids1 = coils.loc[coils['Label']==1,'SID'].tolist()
lsids2 = coils.loc[coils['Label']==2,'SID'].tolist()
#
# Select and extract 30 coil's sids for independent assessemnt
ass1 = random.sample(lsids1, 30)
ass2 = random.sample(lsids2, 30)
res1 = np.array(list(set(lsids1) - set(ass1)))
res2 = np.array(list(set(lsids2) - set(ass2)))
#
# The remaining are organized for building the model
random.shuffle(res1)
random.shuffle(res2)
train1, validate1, test1 = np.split(res1,[int(.7*len(res1)), int(.9*len(res1))])
train2, validate2, test2 = np.split(res2,[int(.7*len(res2)), int(.9*len(res2))])
#
# Building the full sets
train = train1.tolist() + train2.tolist()
valid = validate1.tolist() + validate2.tolist()
test  = test1.tolist()  + test2.tolist()
ass   = ass1   + ass2
#

In [14]:
def featureMap(id, nlcmaps):
    arr1 = nclmaps[id][1234]['nzne'].to_numpy()
    arr2 = nclmaps[id][1243]['nzne'].to_numpy()
    arr  = np.concatenate((arr1, arr2), axis=1)
    return(arr)
#
def prep_dataset(setd,nlcmaps):
    setd_f = []
    setd_l = []
    for i in setd:
        arrimg = featureMap(i, nclmaps)
        lbl = coils.loc[coils['SID']==i,'Label'].values[0]
        setd_f.append(arrimg)
        if lbl == 1:
            setd_l.append([1.,0.])
        if lbl == 2:
            setd_l.append([0.,1.])
    setd_f = np.array(setd_f)
    setd_l = np.array(setd_l)
    return([setd_f,setd_l])
#
def factory_rep(arr,step=0):
    # arr image havind 264 rows and 18 (9 face A+9 face B) columns of normalized data 
    # mirror per face over x axis
    # channels 0-3 => 5-8 and 5-8 => 0-3
    permut1 = list(range(8,4,-1))+[4] + list(range(3,-1,-1))+ list(range(17,13,-1))+[13]+list(range(12,8,-1))
    idx = np.empty_like(permut1)
    idx[permut1] = np.arange(len(permut1))
    arr1 = arr[:,idx]
    permut2 = list(range(263,131,-1))+ list(range(131,-1,-1))
    idx = np.empty_like(permut2)
    idx[permut2] = np.arange(len(permut2))    
    arr2 = arr[idx,:]
    arr3 = arr1[idx,:]
    res  = [arr, arr1, arr2, arr3]
    if step > 0:
        newa = arr
        end = arr.shape[0]-step
        for i in range(arr.shape[0] // step):
            permut = list(range(arr.shape[0]-step,arr.shape[0]))+list(range(0,end))
            idx = np.empty_like(permut)
            idx[permut] = np.arange(len(permut))
            newb= newa[idx,:]
            res.append(newb)
            newa= newb
    res = np.array(res)
    return(res)
#

def prep_dataset_aug(setd,nlcmaps,tlab=-1):
    setd_f = []
    setd_l = []
    for i in setd:
        arrimg = featureMap(i, nclmaps)
        lbl = coils.loc[coils['SID']==i,'Label'].values[0]
        if lbl == tlab: # if lower class => higher augmentation
            res = factory_rep(arrimg,step=8)
        else:
            res = factory_rep(arrimg)
        for j in range(res.shape[0]):
            setd_f.append(res[j,:,:])
            if lbl == 1:
                setd_l.append([1.,0.])
            if lbl == 2:
                setd_l.append([0.,1.])
    setd_f = np.array(setd_f)
    setd_l = np.array(setd_l)
    return([setd_f,setd_l])

In [15]:
i=train[0]
tmpimg = featureMap(i, nclmaps)
tmpimg.shape

(264, 18)

In [16]:
goal = 'C:/Users/alexm/OneDrive/Escritorio/UNIVERSIDAD/TFG/ALVAREZ_CASTRO_Alejandro/ALVAREZ_CASTRO_Alejandro/ZN_1D_imgs/'
# for i in train + valid + test:
train_f, train_l = prep_dataset_aug(train,nclmaps,2)
np.savez(goal+'train.npz', features=train_f, labels=train_l)
#
valid_f, valid_l = prep_dataset(valid,nclmaps)
np.savez(goal+'validation.npz', features=valid_f, labels=valid_l)
#
test_f, test_l = prep_dataset(test,nclmaps)
np.savez(goal+'test.npz', features=test_f, labels=test_l)
#
ass_f, ass_l = prep_dataset(ass,nclmaps)
np.savez(goal+'assess.npz', features=ass_f, labels=ass_l)
#

In [17]:
#
npzfile = np.load(goal+'train.npz')
print(npzfile['features'].shape)
print(npzfile['labels'].shape)

(4293, 264, 18)
(4293, 2)


# Definition of the DQ_Model

In [18]:
class DQCnnNet6(tf.keras.Model):
    """
    - 6 capas convolucionales. Funcion de activación ReLU. 
    - 1 capa de normalización.
    - 3 capas de dropout (0.4 droprate). 
    - 3 capas de average pooling.
    - 2 capas fully connected.
    """

    def __init__(self, inp_shape=(264, 18)):
        super(DQCnnNet6, self).__init__()
        self.inp_shape = inp_shape

        self.kernel_size_0 = 20
        self.kernel_size_1 = 6
        self.drop_rate = 0.4

        self.conv1 = tf.keras.layers.Conv1D(filters=32,
                                            kernel_size=self.kernel_size_0,
                                            activation='relu',
                                            padding="same",
                                            input_shape=self.inp_shape)
        self.conv2 = tf.keras.layers.Conv1D(filters=32,
                                            kernel_size=self.kernel_size_0,
                                            activation='relu',
                                            padding="valid")
        self.conv3 = tf.keras.layers.Conv1D(filters=32,
                                            kernel_size=self.kernel_size_0,
                                            activation='relu',
                                            padding="valid")
        self.conv4 = tf.keras.layers.Conv1D(filters=32,
                                            kernel_size=self.kernel_size_1,
                                            activation='relu',
                                            padding="valid")
        self.conv5 = tf.keras.layers.Conv1D(filters=32,
                                            kernel_size=self.kernel_size_1,
                                            activation='relu',
                                            padding="valid")
        self.conv6 = tf.keras.layers.Conv1D(filters=32,
                                            kernel_size=self.kernel_size_1,
                                            activation='relu',
                                            padding="valid")


        self.batch_n1 = tf.keras.layers.BatchNormalization()

        self.spatial_drop = tf.keras.layers.SpatialDropout1D(self.drop_rate)
        self.avg_pool_1 = tf.keras.layers.MaxPool1D(pool_size=2)
        self.avg_pool_2 = tf.keras.layers.MaxPool1D(pool_size=2)
        self.avg_pool_3 = tf.keras.layers.MaxPool1D(pool_size=2)
        self.avg_pool_4 = tf.keras.layers.MaxPool1D(pool_size=2)
        self.avg_pool_5 = tf.keras.layers.AvgPool1D(pool_size=2)

        self.flat = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(256, activation='relu')
        self.dropout1 = tf.keras.layers.Dropout(self.drop_rate)
        self.dense2 = tf.keras.layers.Dense(148, activation='relu')
        self.dropout2 = tf.keras.layers.Dropout(self.drop_rate)
        self.dense3 = tf.keras.layers.Dense(74, activation='relu')
        self.dropout3 = tf.keras.layers.Dropout(self.drop_rate)

        self.out = tf.keras.layers.Dense(2, activation='softmax')

    def call(self, input_tensor):
        conv1 = self.conv1(input_tensor)
        avg_pool_1 = self.avg_pool_1(conv1)
        batch_n1 = self.batch_n1(avg_pool_1)
        conv2 = self.conv2(batch_n1)
        avg_pool_2 = self.avg_pool_2(conv2)
        conv3 = self.conv3(avg_pool_2)
        batch_n2 = self.batch_n1(conv3)
        conv4 = self.conv4(conv3)
        conv5 = self.conv4(conv4)
        spatial_drop = self.spatial_drop(conv5)
        flat = self.flat(spatial_drop)
        dense1 = self.dense1(flat)
        dropout1 = self.dropout1(dense1)
        dense2 = self.dense2(dropout1)
        dropout2 = self.dropout2(dense2)
        dense3   = self.dense3(dropout2)
        dropout3 = self.dropout3(dense3)
        return self.out(dropout3)


# Training the model

In [19]:
model = DQCnnNet()
num_simulaciones = 15

In [20]:
metrics_plot, conf_plot, mean_precision, std_dev_precision, mean_recall, std_dev_recall, mean_f1, std_dev_f1, mean_accuracy, std_dev_accuracy = simulations(num_simulaciones, model, train_f, train_l, valid_f, valid_l, ass_f, ass_l)

Epoch 1/100
86/86 [==============================] - 8s 67ms/step - loss: 0.6290 - accuracy: 0.6422 - val_loss: 0.7578 - val_accuracy: 0.1746
Epoch 2/100
86/86 [==============================] - 5s 63ms/step - loss: 0.4897 - accuracy: 0.7801 - val_loss: 0.8313 - val_accuracy: 0.1984
Epoch 3/100
86/86 [==============================] - 6s 65ms/step - loss: 0.4329 - accuracy: 0.8253 - val_loss: 0.8077 - val_accuracy: 0.2063
Epoch 4/100
86/86 [==============================] - 5s 64ms/step - loss: 0.3928 - accuracy: 0.8470 - val_loss: 0.6055 - val_accuracy: 0.5159
Epoch 5/100
86/86 [==============================] - 5s 63ms/step - loss: 0.3522 - accuracy: 0.8612 - val_loss: 0.5807 - val_accuracy: 0.5476
Epoch 6/100
86/86 [==============================] - 5s 63ms/step - loss: 0.3388 - accuracy: 0.8691 - val_loss: 0.4856 - val_accuracy: 0.6905
Epoch 7/100
86/86 [==============================] - 5s 63ms/step - loss: 0.3217 - accuracy: 0.8733 - val_loss: 0.5031 - val_accuracy: 0.7222
Epoch 

In [21]:
metrics_plot

In [22]:
conf_plot